# Part 2: Data Pre-processing 
<img src="https://cdn-blog.scalablepath.com/uploads/2021/11/data-preprocessing-techniques-1.png" width="800" height="800">

# Import Dependencies

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Overview of Dataset Preprocessing

<hr>

In order to create more meaningful features from the original `hdb_resale.csv` dataset, we have decided to include relevant data from several sources that will enrich the existing features. The following csv are the data sources we will be utilising:
1. **`hdb_resale_2012_t0_2014.csv`, `hdb_resale_2015_to_2016.csv`, `hdb_resale_2017.csv`** - HDB resale data froom 2012 to 2021
2. **`FivePreventableCrimeCasesRecordedByNeighbourhoodPoliceCentreNPCAnnual.csv`** - Number of crimes recorded by NPC
3. **`address_data_full.csv`** - Data on amenities extracted in 1_Data_Extraction.ipynb

Since we will be using multiple sources and merging them together, there is a need to do basic pre-processing on each dataset individually to ensure data consistency and remove duplicate rows.

## 1) Load and transform HDB resale CSVs
Concatenate the dataset from 2012 to 2021 and remove duplicate records after concatenation. The final csv will be saved as `hdb_resale_2012_onwards.csv`.

In [4]:
#load resale flat price data
resale17 = pd.read_csv("hdb_resale_2017.csv")
resale1516 = pd.read_csv("hdb_resale_2015_to_2016.csv")
resale1214 = pd.read_csv("hdb_resale_2012_t0_2014.csv")

resale_flat_price = pd.concat([resale17, resale1516, resale1214], axis=0)
resale_flat_price.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [5]:
resale_flat_price.info()

<class 'pandas.core.frame.DataFrame'>
Index: 263814 entries, 0 to 52202
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                263814 non-null  object 
 1   town                 263814 non-null  object 
 2   flat_type            263814 non-null  object 
 3   block                263814 non-null  object 
 4   street_name          263814 non-null  object 
 5   storey_range         263814 non-null  object 
 6   floor_area_sqm       263814 non-null  float64
 7   flat_model           263814 non-null  object 
 8   lease_commence_date  263814 non-null  int64  
 9   remaining_lease      211611 non-null  object 
 10  resale_price         263814 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 24.2+ MB


### Drop duplicates after concatenations

In [6]:
# drop all duplicate rows
print(resale_flat_price[resale_flat_price.duplicated(keep='first')])
resale_flat_price = resale_flat_price.drop_duplicates(keep='first')

         month          town flat_type block        street_name storey_range  \
243    2017-01   BUKIT MERAH    4 ROOM   106     HENDERSON CRES     07 TO 09   
305    2017-01  CENTRAL AREA    3 ROOM   271           QUEEN ST     16 TO 18   
510    2017-01   JURONG EAST    4 ROOM   265        TOH GUAN RD     04 TO 06   
591    2017-01   JURONG WEST    4 ROOM  337A       TAH CHING RD     19 TO 21   
672    2017-01     PASIR RIS    4 ROOM   753    PASIR RIS ST 71     01 TO 03   
...        ...           ...       ...   ...                ...          ...   
44758  2014-07      SENGKANG    4 ROOM  272D      SENGKANG CTRL     01 TO 03   
45588  2014-08      CLEMENTI    4 ROOM   347     CLEMENTI AVE 5     10 TO 12   
49618  2014-11         BEDOK    3 ROOM   529     BEDOK NTH ST 3     13 TO 15   
50497  2014-11      SENGKANG    4 ROOM  271C      SENGKANG CTRL     04 TO 06   
52103  2014-12     WOODLANDS    5 ROOM   664  WOODLANDS RING RD     01 TO 03   

       floor_area_sqm         flat_mode

### Change data type of columns and split the columns

In [7]:
#separate year and month column and convert to integers
resale_flat_price[['year', 'month']] = resale_flat_price['month'].str.split('-', expand=True)
resale_flat_price['year'] = resale_flat_price['year'].astype(int)
resale_flat_price['month'] = resale_flat_price['month'].astype(int)
resale_flat_price["address"] = resale_flat_price["block"] + " " + resale_flat_price["street_name"]

resale_flat_price.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year,address
0,1,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017,406 ANG MO KIO AVE 10
1,1,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017,108 ANG MO KIO AVE 4
2,1,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017,602 ANG MO KIO AVE 5
3,1,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,2017,465 ANG MO KIO AVE 10
4,1,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,2017,601 ANG MO KIO AVE 5


### Save the cleaned CSV as `hdb_resale_2012_onwards.csv`.

In [5]:
resale_flat_price.to_csv("hdb_resale_2012_onwards.csv", index=False)

## 2) Load the Crime dataset
Transform the crime dataset so that it can merge with the HDB dataset later on.

In [8]:
#load crimes data
crimes = pd.read_csv("FivePreventableCrimeCasesRecordedByNeighbourhoodPoliceCentreNPCAnnual.csv")
crimes.head()

,year,level_1,value
0,2011,Central Police Division - Total,373
1,2011,Central Police Division - Bukit Merah East NPC,124
2,2011,Central Police Division - Marina Bay NPC,125
3,2011,Central Police Division - Rochor NPC,124
4,2011,Central Police Division - Unknown NPC,0


### Inspect the dataset and the features

In [9]:
resale_flat_price["year"].unique()

array([2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2015, 2016, 2012,
       2013, 2014])

In [10]:
crimes.head()

,year,level_1,value
0,2011,Central Police Division - Total,373
1,2011,Central Police Division - Bukit Merah East NPC,124
2,2011,Central Police Division - Marina Bay NPC,125
3,2011,Central Police Division - Rochor NPC,124
4,2011,Central Police Division - Unknown NPC,0


In [11]:
#shows total number of crimes in each area
crimestotal = crimes[crimes['level_1'].str.endswith('Total')]
crimestotal['town'] = crimestotal['level_1'].str.split('Police').str[0]
crimestotal = crimestotal[['year', 'town', 'value']]
crimestotal = crimestotal.rename(columns={'value': 'crime_count'})
crimestotal.head()

,year,town,crime_count
0,2011,Central,373
5,2011,Clementi,380
11,2011,Tanglin,423
18,2011,Ang Mo Kio,625
29,2011,Bedok,830


In [12]:
crimes["level_1"].unique()

array(['Central Police Division - Total',
       'Central Police Division - Bukit Merah East NPC',
       'Central Police Division - Marina Bay NPC',
       'Central Police Division - Rochor NPC',
       'Central Police Division - Unknown NPC',
       'Clementi Police Division - Total',
       'Clementi Police Division - Bukit Merah West NPC',
       'Clementi Police Division - Clementi NPC',
       'Clementi Police Division - Jurong East NPC',
       'Clementi Police Division - Queenstown NPC',
       'Clementi Police Division - Unknown NPC',
       'Tanglin Police Division - Total',
       'Tanglin Police Division - Bishan NPC',
       'Tanglin Police Division - Bukit Timah NPC',
       'Tanglin Police Division - Kampong Java NPC',
       'Tanglin Police Division - Orchard NPC',
       'Tanglin Police Division - Toa Payoh NPC',
       'Tanglin Police Division - Unknown NPC',
       'Ang Mo Kio Police Division - Total',
       'Ang Mo Kio Police Division - Ang Mo Kio North NPC',
     

In [13]:
crimes["year"].unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

### Extract out the names of the NPC and drop irrelevant columns 

In [14]:
#extract town and specific areas from level_1 col
crimes2 = crimes[~crimes['level_1'].str.endswith('Total')]
crimes2['town'] = crimes2['level_1'].str.split('Police').str[0]
crimes2['town'] = crimes2['town'].str.strip()

def extract_substring(row):
    start_index = row.find('- ')
    if start_index != -1:
        return row[start_index + 1:].strip()
    else:
        return None 
    
crimes2['NPC'] = crimes2['level_1'].apply(extract_substring)
crimes2 = crimes2.drop(crimes2[crimes2["NPC"] == "Unknown NPC"].index)
crimes2 = crimes2.drop(crimes2[crimes2["value"]=="na"].index)
crimes2 = crimes2.drop(["level_1", "town"], axis=1)
crimes2 = crimes2.drop_duplicates()
crimes2 = crimes2.rename({"value":"crimes_recorded"}, axis=1)
crimes2.groupby(["NPC", "year"]).sum().reset_index()
crimes2.head()

,year,crimes_recorded,NPC
1,2011,124,Bukit Merah East NPC
2,2011,125,Marina Bay NPC
3,2011,124,Rochor NPC
6,2011,135,Bukit Merah West NPC
7,2011,73,Clementi NPC


## 3) Load the address data CSV
Transform the columns as needed so that we can merge the dataset with the crime dataset.

In [15]:
address_info = pd.read_csv("address_data_full.csv")
address_info.head()

,Search Name,Address,Address Lat,Address Long,Nearest MRT Station,Nearest MRT Distance,Nearest Mall Name,Nearest Mall Distance,Nearest Bus Stop Code,Nearest Bus Stop Distance,Number of Bus Stops,Nearest NPC,Nearest NPC Distance,Nearest School,Nearest School Distance,Nearest Green Area,Nearest Green Area Distance
0,406 ANG MO KIO AVE 10,406 ANG MO KIO AVENUE 10,1.362005,103.853880,ANG MO KIO MRT STATION (NS16),1.003997,CATHAY CINEPLEX AMK HUB,1.000654,54319 (BUS STOP),0.091925,2,Bishan Neighbourhood Police Centre,0.829747,Townsville Primary School,0.219348,Bishan-Ang Mo Kio Park,0.693469
1,108 ANG MO KIO AVE 4,108 ANG MO KIO AVENUE 4,1.370966,103.838202,MAYFLOWER MRT STATION (TE6),0.189875,CATHAY CINEPLEX AMK HUB,1.157763,54189 (BUS STOP),0.166313,3,Ang Mo Kio South Neighbourhood Police Centre,1.506198,Ang Mo Kio Primary School,0.242224,Ang Mo Kio Town Garden West,0.636510
2,602 ANG MO KIO AVE 5,602 ANG MO KIO AVENUE 5,1.380709,103.835368,LENTOR MRT STATION (TE5),0.535117,CATHAY CINEPLEX AMK HUB,1.938852,55121 (BUS STOP),0.123479,2,Ang Mo Kio North Neighbourhood Police Centre,1.182304,Anderson Primary School,0.777627,Ang Mo Kio Town Garden West,1.113168
3,465 ANG MO KIO AVE 10,465 ANG MO KIO AVENUE 10,1.366201,103.857201,ANG MO KIO MRT STATION (NS16),0.945529,MYVILLAGE AT SERANGOON GARDEN,0.892904,54389 (BUS STOP),0.069453,2,Ang Mo Kio South Neighbourhood Police Centre,0.737154,Deyi Secondary School,0.517985,Ang Mo Kio Town Garden East,0.898934
4,601 ANG MO KIO AVE 5,601 ANG MO KIO AVENUE 5,1.381041,103.835132,LENTOR MRT STATION (TE5),0.501151,CATHAY CINEPLEX AMK HUB,1.983038,55121 (BUS STOP),0.155488,3,Ang Mo Kio North Neighbourhood Police Centre,1.192218,Anderson Primary School,0.782772,Ang Mo Kio Town Garden West,1.157278


### Ensure that the value of the primary column in both dataset have the same naming convention

In [16]:
address_info["Nearest NPC"] = address_info["Nearest NPC"].str.replace("Neighbourhood Police Centre", "NPC")

In [17]:
address_info.head()

,Search Name,Address,Address Lat,Address Long,Nearest MRT Station,Nearest MRT Distance,Nearest Mall Name,Nearest Mall Distance,Nearest Bus Stop Code,Nearest Bus Stop Distance,Number of Bus Stops,Nearest NPC,Nearest NPC Distance,Nearest School,Nearest School Distance,Nearest Green Area,Nearest Green Area Distance
0,406 ANG MO KIO AVE 10,406 ANG MO KIO AVENUE 10,1.362005,103.853880,ANG MO KIO MRT STATION (NS16),1.003997,CATHAY CINEPLEX AMK HUB,1.000654,54319 (BUS STOP),0.091925,2,Bishan NPC,0.829747,Townsville Primary School,0.219348,Bishan-Ang Mo Kio Park,0.693469
1,108 ANG MO KIO AVE 4,108 ANG MO KIO AVENUE 4,1.370966,103.838202,MAYFLOWER MRT STATION (TE6),0.189875,CATHAY CINEPLEX AMK HUB,1.157763,54189 (BUS STOP),0.166313,3,Ang Mo Kio South NPC,1.506198,Ang Mo Kio Primary School,0.242224,Ang Mo Kio Town Garden West,0.636510
2,602 ANG MO KIO AVE 5,602 ANG MO KIO AVENUE 5,1.380709,103.835368,LENTOR MRT STATION (TE5),0.535117,CATHAY CINEPLEX AMK HUB,1.938852,55121 (BUS STOP),0.123479,2,Ang Mo Kio North NPC,1.182304,Anderson Primary School,0.777627,Ang Mo Kio Town Garden West,1.113168
3,465 ANG MO KIO AVE 10,465 ANG MO KIO AVENUE 10,1.366201,103.857201,ANG MO KIO MRT STATION (NS16),0.945529,MYVILLAGE AT SERANGOON GARDEN,0.892904,54389 (BUS STOP),0.069453,2,Ang Mo Kio South NPC,0.737154,Deyi Secondary School,0.517985,Ang Mo Kio Town Garden East,0.898934
4,601 ANG MO KIO AVE 5,601 ANG MO KIO AVENUE 5,1.381041,103.835132,LENTOR MRT STATION (TE5),0.501151,CATHAY CINEPLEX AMK HUB,1.983038,55121 (BUS STOP),0.155488,3,Ang Mo Kio North NPC,1.192218,Anderson Primary School,0.782772,Ang Mo Kio Town Garden West,1.157278


### Merge all 3 datasets together and drop rows with null values 

In [18]:
hdb_resale_full = resale_flat_price.merge(address_info, left_on="address", right_on="Search Name", how="left")

# drop years after 2021 since we only have crime data until 2021
hdb_resale_full = hdb_resale_full.drop(hdb_resale_full[hdb_resale_full['year']>2021].index)
hdb_resale_full = hdb_resale_full.merge(crimes2, left_on=["Nearest NPC", "year"], right_on=["NPC", "year"], how="left")
hdb_resale_full = hdb_resale_full.dropna(subset=["Address"])

# Assume NPC with no crime value recorded as 0
hdb_resale_full = hdb_resale_full.fillna(value={"crimes_recorded":0})
hdb_resale_full = hdb_resale_full.drop(["NPC","address","Search Name"], axis=1)

print(hdb_resale_full.info())

hdb_resale_full.head()

<class 'pandas.core.frame.DataFrame'>
Index: 208389 entries, 0 to 208392
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   month                        208389 non-null  int64  
 1   town                         208389 non-null  object 
 2   flat_type                    208389 non-null  object 
 3   block                        208389 non-null  object 
 4   street_name                  208389 non-null  object 
 5   storey_range                 208389 non-null  object 
 6   floor_area_sqm               208389 non-null  float64
 7   flat_model                   208389 non-null  object 
 8   lease_commence_date          208389 non-null  int64  
 9   remaining_lease              156438 non-null  object 
 10  resale_price                 208389 non-null  float64
 11  year                         208389 non-null  int64  
 12  Address                      208389 non-null  object 
 13  Addr

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,Nearest Bus Stop Code,Nearest Bus Stop Distance,Number of Bus Stops,Nearest NPC,Nearest NPC Distance,Nearest School,Nearest School Distance,Nearest Green Area,Nearest Green Area Distance,crimes_recorded
0,1,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,...,54319 (BUS STOP),0.091925,2.0,Bishan NPC,0.829747,Townsville Primary School,0.219348,Bishan-Ang Mo Kio Park,0.693469,29
1,1,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,...,54189 (BUS STOP),0.166313,3.0,Ang Mo Kio South NPC,1.506198,Ang Mo Kio Primary School,0.242224,Ang Mo Kio Town Garden West,0.636510,35
2,1,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,55121 (BUS STOP),0.123479,2.0,Ang Mo Kio North NPC,1.182304,Anderson Primary School,0.777627,Ang Mo Kio Town Garden West,1.113168,34
3,1,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,...,54389 (BUS STOP),0.069453,2.0,Ang Mo Kio South NPC,0.737154,Deyi Secondary School,0.517985,Ang Mo Kio Town Garden East,0.898934,35
4,1,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,55121 (BUS STOP),0.155488,3.0,Ang Mo Kio North NPC,1.192218,Anderson Primary School,0.782772,Ang Mo Kio Town Garden West,1.157278,34


## 4) Save the final dataset
This dataset includes the full range of features we have extracted from different sources and have been cleaned for downstream analysis and model training.

In [23]:
hdb_resale_full.to_csv("hdb_resale_full.csv", index=False)